In [3]:
import numpy as np
import pyxdf
import pandas as pd
import mne
import mne_connectivity
import matplotlib
import matplotlib.pyplot as plt

from scipy.signal import sosfiltfilt, butter, hilbert
from scipy import io
from scipy.stats import pearsonr
from scipy.io import loadmat

from pingouin import circ_corrcl

import seaborn as sns

import neurodsp
from neurodsp import spectral
from neurodsp import filt
from neurodsp import sim

import fooof
from fooof import FOOOF
from fooof import FOOOFGroup
#from fooof.plts.templates import plot_spectrum
from sklearn.decomposition import PCA
from mne.datasets import eegbci, fetch_fsaverage
import os.path as op
import ipywidgets
from ipywidgets import Image
import pyvistaqt
import nibabel
import trame
import vtk
import vtkmodules


In [4]:

# Download fsaverage files
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)

# The files live in:
subject = "fsaverage"
trans = "fsaverage"  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")

0 files missing from root.txt in C:\Users\jaket\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\jaket\mne_data\MNE-fsaverage-data\fsaverage


In [5]:
data = np.load(r'C:\Users\jaket\COGS280\DataDeco\eeg.npy')

data1 = data[0:10,:,:]  #just the first 10 trials and make them epochs, next cell will make evoked data

channel_names = mne.channels.make_standard_montage('biosemi64').ch_names
channel_locs = mne.channels.make_standard_montage('biosemi64').get_positions()['ch_pos']

info = mne.create_info(channel_names, sfreq=128, ch_types='eeg')  # Adjust accordingly
#raw = mne.io.RawArray(data1, info, verbose = False)
epochs = mne.EpochsArray(data1, info)

# Set montage if known
montage = mne.channels.make_standard_montage('biosemi64')
epochs.set_montage(montage)

epochs.set_eeg_reference(projection=True)  # needed for inverse modeling


Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


Number of events,10
Events,1: 10
Time range,0.000 – 1.289 s
Baseline,off


In [6]:
print(data[0,0,0])

-0.5211273451351086


In [7]:
fwd = mne.make_forward_solution(
    epochs.info, trans=trans, src=src, bem=bem, eeg=True, mindist=5.0
)
print(fwd)

Source space          : C:\Users\jaket\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif
MRI -> head transform : C:\Users\jaket\AppData\Roaming\Python\Python39\site-packages\mne\data\fsaverage\fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : C:\Users\jaket\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading C:\Users\jaket\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif...
Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999994  0.003552  0.000202      -1.76 mm
    -0.003558  0.998389  0.056626      31.09 mm
    -0.000001 -0.056626  0.998395      39.60 mm
     0.000000  0.000000  0.000000       1.00

Read  64 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Setting up

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found     0/ 7809 points outside using solid angles
    Total 10242/10242 points inside the surface
Interior check completed in 17047.2 ms
Checking surface interior status for 10242 points...
    Found  2241/10242 points inside  an interior sphere of radius   47.7 mm
    Found     0/10242 points outside an exterior sphere of radius   98.3 mm
    Found     0/ 8001 points outside using surface Qhull


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found     0/ 8001 points outside using solid angles
    Total 10242/10242 points inside the surface
Interior check completed in 15249.3 ms



...

Setting up for EEG...
Computing EEG at 20484 source locations (free orientations)...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



Finished.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.7s finished


<Forward | MEG channels: 0 | EEG channels: 64 | Source space: Surface with 20484 vertices | Source orientation: Free>


In [8]:
from mne import compute_covariance

# Assuming you have an 'epochs' object
cov = compute_covariance(epochs, tmin=0.5)

from mne.minimum_norm import make_inverse_operator

inverse_operator = make_inverse_operator(epochs.info, fwd, cov, loose=0.2, depth=0.8)

Computing rank from data with rank=None


C:\Users\jaket\AppData\Local\Temp\ipykernel_9396\3490624686.py:4: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  cov = compute_covariance(epochs, tmin=0.5)


    Using tolerance 5.1e-06 (2.2e-16 eps * 64 dim * 3.6e+08  max singular value)
    Estimated rank (eeg): 63
    EEG: rank 63 computed from 64 data channels with 1 projector
    Created an SSP operator (subspace dimension = 1)
    Setting small EEG eigenvalues to zero (without PCA)
Reducing data rank from 64 -> 63
Estimating covariance using EMPIRICAL
Done.
Number of samples used : 1020
[done]
Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 64 channels.
    64 out of 64 channels remain after picking
Selected 64 channels
Creating the depth weighting matrix...
    64 EEG channels
    limit = 20485/20484 = 2.329337
    scale = 108975 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP 

In [9]:
import pyvistaqt

# Average the epochs to create evoked data
evoked = epochs.average()

# Apply the inverse operator to the evoked data
from mne.minimum_norm import apply_inverse

# Regularization parameter
lambda2 = 1.0 / 9.0

# Method for source estimation, e.g., dSPM, sLORETA
method = 'dSPM'

stc = apply_inverse(evoked, inverse_operator, lambda2, method=method, pick_ori=None)

brain = stc.plot(
    hemi='both', subjects_dir=subjects_dir, subject=subject,
    initial_time=0, clim=dict(kind='value', lims=[0.3, 0.6, 0.9]))

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 10
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 62 (2 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Applying inverse operator to "1"...
    Picked 64 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained  83.6% variance
    Combining the current components...
    dSPM...
[done]
Using pyvistaqt 3d backend.

For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
For automatic theme 

In [13]:

print(mne.channels.get_builtin_montages())
  #just the first 10 trials and make them epochs, next cell will make evoked data

channel_names = mne.channels.make_standard_montage('standard_1020').ch_names
channel_locs = mne.channels.make_standard_montage('standard_1020').get_positions()['ch_pos']

print(channel_names)

raw_EO = mne.io.read_raw_eeglab('MichaelEOxdf.set')
print(raw_EO.ch_names)

['standard_1005', 'standard_1020', 'standard_alphabetic', 'standard_postfixed', 'standard_prefixed', 'standard_primed', 'biosemi16', 'biosemi32', 'biosemi64', 'biosemi128', 'biosemi160', 'biosemi256', 'easycap-M1', 'easycap-M10', 'easycap-M43', 'EGI_256', 'GSN-HydroCel-32', 'GSN-HydroCel-64_1.0', 'GSN-HydroCel-65_1.0', 'GSN-HydroCel-128', 'GSN-HydroCel-129', 'GSN-HydroCel-256', 'GSN-HydroCel-257', 'mgh60', 'mgh70', 'artinis-octamon', 'artinis-brite23', 'brainproducts-RNP-BA-128']
['Fp1', 'Fpz', 'Fp2', 'AF9', 'AF7', 'AF5', 'AF3', 'AF1', 'AFz', 'AF2', 'AF4', 'AF6', 'AF8', 'AF10', 'F9', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'F10', 'FT9', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'FT10', 'T9', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'T10', 'TP9', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'TP10', 'P9', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO9', 'PO7', 'PO5', 'PO3', 'PO1', 'POz', 'PO2', 'PO4', 'P

C:\Users\jaket\AppData\Local\Temp\ipykernel_9396\1386082838.py:9: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw_EO = mne.io.read_raw_eeglab('MichaelEOxdf.set')


In [26]:
# Define your subset of electrodes
montage_1020 = mne.channels.make_standard_montage('standard_1020')
channels_1020 = [item.lower() for item in montage_1020.ch_names]
subset_electrodes = [item.lower() for item in raw_EO.ch_names]

montage_1020.ch_names = channels_1020
# Filter the montage for the subset
subset_montage = montage_1020.copy()  # Copy the original montage

subset_montage.ch_names = subset_electrodes  # Keep only the subset

# Get the positions of all electrodes in the montage
all_positions = montage_1020.get_positions()
# Filter for the subset
subset_positions = {ch: all_positions['ch_pos'][ch] for ch in subset_electrodes}
# Update the positions to only include the subset

# Create a new montage from the subset positions
subset_montage = mne.channels.make_dig_montage(ch_pos=subset_positions, coord_frame='head')

info = mne.create_info(subset_electrodes, sfreq=256, ch_types='eeg')  # Adjust accordingly

raw_EO.rename_channels({ch: ch.lower() for ch in raw_EO.ch_names})
raw_EO.set_montage(subset_montage)

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,22 points
Good channels,19 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,256.00 Hz
Highpass,0.00 Hz
Lowpass,128.00 Hz


In [29]:
# Example of manually creating an events array
# Each event is defined as [sample index, 0, event id]
events = np.array([[100, 0, 1], [200, 0, 2], [300, 0, 1]])

# Event id dictionary (optional, for labeling)
event_id = {'Stimulus/Type1': 1, 'Stimulus/Type2': 2}

# Create epochs
epochs = mne.Epochs(raw_EO, events, event_id=event_id, tmin=-0.1, tmax=0.3, preload=True)

epochs.set_montage(subset_montage)

epochs.set_eeg_reference(projection=True)  # needed for inverse modeling
epochs.apply_proj()

Not setting metadata
3 matching events found
Setting baseline interval to [-0.1015625, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 3 events and 104 original time points ...
0 bad epochs dropped
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


Number of events,3
Events,Stimulus/Type1: 2Stimulus/Type2: 1
Time range,-0.102 – 0.301 s
Baseline,-0.102 – 0.000 s


In [30]:
fwd = mne.make_forward_solution(
    epochs.info, trans=trans, src=src, bem=bem, eeg=True, mindist=5.0
)
print(fwd)

Source space          : C:\Users\jaket\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif
MRI -> head transform : C:\Users\jaket\AppData\Roaming\Python\Python39\site-packages\mne\data\fsaverage\fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : C:\Users\jaket\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading C:\Users\jaket\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif...
Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999994  0.003552  0.000202      -1.76 mm
    -0.003558  0.998389  0.056626      31.09 mm
    -0.000001 -0.056626  0.998395      39.60 mm
     0.000000  0.000000  0.000000       1.00

Read  19 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Setting up

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found     0/ 7809 points outside using solid angles
    Total 10242/10242 points inside the surface
Interior check completed in 8945.9 ms
Checking surface interior status for 10242 points...
    Found  2241/10242 points inside  an interior sphere of radius   47.7 mm
    Found     0/10242 points outside an exterior sphere of radius   98.3 mm
    Found     0/ 8001 points outside using surface Qhull


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found     0/ 8001 points outside using solid angles
    Total 10242/10242 points inside the surface
Interior check completed in 8334.2 ms



[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.2s finished


Setting up for EEG...
Computing EEG at 20484 source locations (free orientations)...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



Finished.
<Forward | MEG channels: 0 | EEG channels: 19 | Source space: Surface with 20484 vertices | Source orientation: Free>


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.7s finished


In [32]:
from mne import compute_covariance

# Assuming you have an 'epochs' object
cov = compute_covariance(epochs, tmin=0.2)

from mne.minimum_norm import make_inverse_operator

inverse_operator = make_inverse_operator(epochs.info, fwd, cov, loose=0.2, depth=0.8)

Computing rank from data with rank=None
    Using tolerance 6.3e-13 (2.2e-16 eps * 19 dim * 1.5e+02  max singular value)
    Estimated rank (eeg): 18
    EEG: rank 18 computed from 19 data channels with 1 projector
    Created an SSP operator (subspace dimension = 1)
    Setting small EEG eigenvalues to zero (without PCA)
Reducing data rank from 19 -> 18
Estimating covariance using EMPIRICAL
Done.
Number of samples used : 81
[done]
Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...


C:\Users\jaket\AppData\Local\Temp\ipykernel_9396\3849480291.py:4: RuntimeWarning: Too few samples (required : 100 got : 81), covariance estimate may be unreliable
  cov = compute_covariance(epochs, tmin=0.2)


    [done]
Computing inverse operator with 19 channels.
    19 out of 19 channels remain after picking
Selected 19 channels
Creating the depth weighting matrix...
    19 EEG channels
    limit = 20485/20484 = 2.259049
    scale = 40062.8 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 1.2e-14 (2.2e-16 eps * 19 dim * 2.8  max singular value)
    Estimated rank (eeg): 18
    EEG: rank 18 computed from 19 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.
    largest singular value = 2.59981
    scaling factor to adjust the trace = 1.68627e+20 (nchan = 19 nzero = 1)


In [33]:
import pyvistaqt

# Average the epochs to create evoked data
evoked = epochs.average()

# Apply the inverse operator to the evoked data
from mne.minimum_norm import apply_inverse

# Regularization parameter
lambda2 = 1.0 / 9.0

# Method for source estimation, e.g., dSPM, sLORETA
method = 'dSPM'

stc = apply_inverse(evoked, inverse_operator, lambda2, method=method, pick_ori=None)

brain = stc.plot(
    hemi='both', subjects_dir=subjects_dir, subject=subject,
    initial_time=0, clim=dict(kind='value', lims=[0.3, 0.6, 0.9]))

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 3
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 18 (1 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Applying inverse operator to "0.67 × Stimulus/Type1 + 0.33 × Stimulus/Type2"...
    Picked 19 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained  79.5% variance
    Combining the current components...
    dSPM...
[done]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
For au